<a href="https://colab.research.google.com/github/luis-arrieta/Building-Generative-AI-Powered-Applications-with-Python/blob/main/Give_Meaningful_Names_to_Your_Photos_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction ###


Images, rich with untapped information, often come under the radar of search engines and data systems. Transforming this visual data into machine-readable language is no easy task, but it's where image captioning AI is useful. Here's how image captioning AI can make a difference:

* Improves accessibility: Helps visually impaired individuals understand visual content.
* Enhances SEO: Assists search engines in identifying the content of images.
* Facilitates content discovery: Enables efficient analysis and categorization of large image databases.
* Supports social media and advertising: Automates engaging description generation for visual content.
* Boosts security: Provides real-time descriptions of activities in video footage.
* Aids in education and research: Assists in understanding and interpreting visual materials.
* Offers multilingual support: Generates image captions in various languages for international audiences.
* Enables data organization: Helps manage and categorize large sets of visual data.
* Saves time: Automated captioning is more efficient than manual efforts.
* Increases user engagement: Detailed captions can make visual content more engaging and informative.

### Learning objectives ###

At the end of this project, you will be able to:

* Implement an image captioning tool using the BLIP model from Hugging Face's Transformers

* Use Gradio to provide a user-friendly interface for your image captioning application

* Adapt the tool for real-world business scenarios, demonstrating its practical applications

In [ ]:
!pip install virtualenv

In [ ]:
!virtualenv my_env # create a virtual environment my_env

In [3]:
!source my_env/bin/activate # activate my_env

In [ ]:
!pip uninstall numpy
!pip install numpy

In [ ]:
# installing required libraries in my_env
!pip install langchain gradio==5.23.2 transformers==4.41.0 bs4==0.0.2 requests==2.32.3 torch==2.6.0

In [6]:
import requests
from PIL import Image
from transformers import AutoProcessor, BlipForConditionalGeneration
# Load the pretrained processor and model
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# Load your image, DONT FORGET TO WRITE YOUR IMAGE NAME
img_path = "lion.jpg"
# convert it into an RGB format
image = Image.open(img_path).convert('RGB')

In [8]:
# Generate a caption for the image
inputs = processor(image, return_tensors="pt")
outputs = model.generate(**inputs, max_length=50)

In [9]:
# Decode the generated tokens to text
caption = processor.decode(outputs[0], skip_special_tokens=True)
# Print the caption
print(caption)

a lion laying in the grass


### Image captioning app with Gradio ###

In [10]:
import gradio as gr
def greet(name):
    return "Hello " + name + "!"
demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch(server_name="0.0.0.0", server_port= 7860)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1186ca01df9fee75ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Implement image captioning app with Gradio ###

In [12]:
import requests
from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup
from transformers import AutoProcessor, BlipForConditionalGeneration
# Load the pretrained processor and model
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
# URL of the page to scrape
url = "https://en.wikipedia.org/wiki/IBM"
# Download the page
response = requests.get(url)
# Parse the page with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
# Find all img elements
img_elements = soup.find_all('img')
# Open a file to write the captions
with open("captions.txt", "w") as caption_file:
    # Iterate over each img element
    for img_element in img_elements:
        img_url = img_element.get('src')
        # Skip if the image is an SVG or too small (likely an icon)
        if 'svg' in img_url or '1x1' in img_url:
            continue
        # Correct the URL if it's malformed
        if img_url.startswith('//'):
            img_url = 'https:' + img_url
        elif not img_url.startswith('http://') and not img_url.startswith('https://'):
            continue  # Skip URLs that don't start with http:// or https://
        try:
            # Download the image
            response = requests.get(img_url)
            # Convert the image data to a PIL Image
            raw_image = Image.open(BytesIO(response.content))
            if raw_image.size[0] * raw_image.size[1] < 400:  # Skip very small images
                continue
            raw_image = raw_image.convert('RGB')
            # Process the image
            inputs = processor(raw_image, return_tensors="pt")
            # Generate a caption for the image
            out = model.generate(**inputs, max_new_tokens=50)
            # Decode the generated tokens to text
            caption = processor.decode(out[0], skip_special_tokens=True)
            # Write the caption to the file, prepended by the image URL
            caption_file.write(f"{img_url}: {caption}\n")
        except Exception as e:
            print(f"Error processing image {img_url}: {e}")
            continue

### Image captioning for local files (Run locally if using Blip2) ###

In [ ]:
import os
import glob
import requests
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration #Blip2 models
# Load the pretrained processor and model
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b")
# Specify the directory where your images are
image_dir = "/path/to/your/images"
image_exts = ["jpg", "jpeg", "png"]  # specify the image file extensions to search for
# Open a file to write the captions
with open("captions.txt", "w") as caption_file:
    # Iterate over each image file in the directory
    for image_ext in image_exts:
        for img_path in glob.glob(os.path.join(image_dir, f"*.{image_ext}")):
            # Load your image
            raw_image = Image.open(img_path).convert('RGB')
            # You do not need a question for image captioning
            inputs = processor(raw_image, return_tensors="pt")
            # Generate a caption for the image
            out = model.generate(**inputs, max_new_tokens=50)
            # Decode the generated tokens to text
            caption = processor.decode(out[0], skip_special_tokens=True)
            # Write the caption to the file, prepended by the image file name
            caption_file.write(f"{os.path.basename(img_path)}: {caption}\n")